# 협업필터링(CF) 추천 - 기본
- 협업 필터링 (Collaborative filtering: CF)

In [1]:
import numpy as np
print(f'NumPy v{np.__version__}')

import pandas as pd
print(f'pandas v{pd.__version__}')

import sklearn
print(f'scikit-learn v{sklearn.__version__}')

# Only for specifying versions
import sys; print(f'Python v{sys.version}')

NumPy v1.25.0
pandas v1.5.3
scikit-learn v1.2.0
Python v3.9.16 (main, May 17 2023, 17:49:16) [MSC v.1916 64 bit (AMD64)]


## 3.1 협업 필터링의 원리

협업 필터링(CF)은 기본적으로 취향이 비슷한 사람들의 집단이 존재한다고 가정한다. 추천의 대상이 되는 어떤 한 사람이 있으면, 이 사람과 취향이 비슷한 사람들(neighbor)을 찾아내기만 하면 이 사람들이 공통적으로 좋아하는 제품이나 서비스를 추천 대상인 사람에게 추천하면 된다는 것이 기본 아이디어이다.

## 3.3 기본 CF 알고리즘

아래의 cell은 `2-2.ipynb`의 2~4번째 cell과 동일한 코드이다.

In [2]:
# 데이터 읽어 오기
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('../Data/u.item', sep='|', names=i_cols, encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# timestamp 제거 
ratings.drop('timestamp', axis='columns', inplace=True)

# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

가장 기본적인 CF 알고리즘은 전체 사용자를 이웃(neighbor)으로 설정한다. 즉 현재 사용자와 취향이 비슷한 사용자 그룹을 따로 선정하지 않고 모든 사용자의 평점을 가지고 예측을 한다. 구체적으로는 아래와 같이 작동한다.
1. 모든 사용자 간의 평가의 유사도를 계산한다. 상관계수(correlation coefficient), 코사인 유사도 등을 사용할 수 있다.
2. 쳔재 추천 대상이 되는 사람과 다른 사용자의 유사도를 추출한다.
3. 현재 사용자가 평가하지 않은 모든 아이템에 대해서 현재 사용자의 예상 평가값을 구한다. 예상 평가값은 다른 사용자의 해당 아이템에 대한 평가(평점)를 현재 사용자와 그 사용자의 유사도로 가중해서 평균을 낸다.
4. 아이템 중에서 예상 평가값이 가장 높은 N개의 아이템을 추천한다.

In [3]:
# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity

u_id = rating_matrix.index
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=u_id, columns=u_id)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.108361,0.046638,0.029577,0.245753,0.335853,0.344724,0.191582,0.057149,0.251979,...,0.257073,0.069412,0.231643,0.108093,0.176842,0.104799,0.232472,0.051528,0.129555,0.256333
2,0.108361,1.000000,0.057613,0.130237,0.054918,0.190552,0.079399,0.076146,0.167992,0.147376,...,0.136993,0.252887,0.255454,0.285193,0.232751,0.149088,0.102807,0.062386,0.109143,0.107686
3,0.046638,0.057613,1.000000,0.139805,0.000000,0.032485,0.043869,0.080968,0.022263,0.059925,...,0.027402,0.000000,0.175060,0.010343,0.105635,0.019052,0.127099,0.023917,0.060392,0.000000
4,0.029577,0.130237,0.139805,1.000000,0.000000,0.045190,0.088586,0.199526,0.135013,0.026919,...,0.055392,0.049773,0.076549,0.139382,0.113886,0.000000,0.130343,0.077357,0.157890,0.063911
5,0.245753,0.054918,0.000000,0.000000,1.000000,0.176443,0.281860,0.132205,0.038790,0.134200,...,0.183969,0.019305,0.073714,0.041807,0.081088,0.029743,0.188392,0.068342,0.055557,0.207259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.104799,0.149088,0.019052,0.000000,0.029743,0.086464,0.075012,0.095736,0.000000,0.080883,...,0.061061,0.299811,0.158064,0.221251,0.323989,1.000000,0.047368,0.162173,0.058828,0.124548
940,0.232472,0.102807,0.127099,0.130343,0.188392,0.230566,0.270071,0.164157,0.131458,0.255758,...,0.195863,0.113346,0.144570,0.173568,0.139877,0.047368,1.000000,0.092911,0.199881,0.135868
941,0.051528,0.062386,0.023917,0.077357,0.068342,0.095478,0.020036,0.076269,0.106763,0.063461,...,0.021901,0.055348,0.226017,0.170493,0.249612,0.162173,0.092911,1.000000,0.072402,0.099200


아래 `CF_simple()`에서 사용자 `u`의 영화 `M`에 대한 점수를 예측할 때, `M`에 점수를 매긴 사용자 집단 `{u_i}`와 `u`의 cosine similarity가 모두 0인 경우가 있을 수 있다. 이는 `u`와 `u_i`들이 공통적으로 본 영화가 `M` 뿐인데, 이 (`u`,`M`) pair가 test set에 들어가버려서 train set을 기준으로 구한 cosine similiarity가 0으로 계산되는 것이다.

이 코드에서는, 사용자 `914`의 영화 `1259`에 대한 점수를 예측할 때, train set에서 `M`에 점수를 매긴 사용자는 `181` 뿐이었는데 (1점) `914`와 `181`의 cosine similarity가 0인 것으로 관측된다.

이런 경우 weighted average를 구하기 위해 `sim_scores.sum()`으로 나눌 때 `Zero-Division Error`가 생기므로, weight 없이 단순 average로 예측하도록 하겠다.

In [4]:
# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()

        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()

        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index

        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings.dropna(inplace=True)      # movie_ratings.drop(none_rating_idx, inplace=True)와 같다.
        # 현재 영화를 평가하지 않은 사용자와의 similarity값 제거
        sim_scores.drop(none_rating_idx, inplace=True)

        if sim_scores.sum() != 0:
            # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        else:
            mean_rating = movie_ratings.mean()
        
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_simple)

1.023642893837376

단순 CF 알고리즘으로 예측치를 계산한 경우 RMSE가 1.024로 `2-2.ipynb`의 성별 모델(1.039)이나 직업 모델(1.121)에 비해 소폭 개선되었다고 할 수 있다. (참고) Best-seller 모델은 1.030)

즉 평가경향이 비슷한 (유사도가 높은) 사용자에게 가중치를 더 주어서 평균을 계산하면 더 정확한 예측을 할 수 있음을 알 수 있다.

## 연습문제

Q. 코사인 유사도 대신에 피어슨 상관계수 (Pearson correlation coefficient)를 사용하는 코드를 작성하고 RMSE를 계산하라.

In [5]:
user_similarity = matrix_dummy.corr()
user_similarity = pd.DataFrame(user_similarity, index=u_id, columns=u_id)
user_similarity += 1    # Corr coef ranges from -1 to 1.

score(CF_simple)

1.0300938432316478